In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
plt.rcParams["animation.html"] = "jshtml"
import numpy as np
from random import random

from matplotlib.animation import FuncAnimation
from IPython.display import HTML
#%matplotlib notebook

In [2]:
%%capture  
# prevent the output of a cell in jupyter notebook

# prevent the output of a cell in jupyter notebook
%matplotlib inline

# Holonomic version
# works, don't change!!!!!




# robot is operated as material point
# x, y coordinates and orientation theta of robot in WCF are known
#  x_goal, y_goal, theta_goal

# At each timestep, the robot finds the proper lookahead point, 
# calculates the curvature and velocity needed to get to that point, 


#def initial_poses():
# np.random.seed(12)
# x_start = 5 * random()
# y_start = 5 * random()
# theta_start = 2 * np.pi * np.random.rand() - np.pi
# x_goal = 15 * random()
# y_goal = 15 * random()
# theta_goal = 2 * np.pi * random() - np.pi/2


x_start = 10
y_start = 12
theta_start = np.pi/2
x_goal = 4
y_goal = 2
theta_goal = 3*np.pi/2


x_traj, y_traj, theta_hist = [], [], []
x_traj.append(x_start)
y_traj.append(y_start)
theta_hist.append(theta_start)
    
    #return x_goal, y_goal, theta_goal, x_traj, y_traj, theta_hist

Kp_rho = 5
dt = 0.01
#w = 1.5

# create an output with an empty figure
fig, ax1 = plt.subplots(figsize=(6, 6))
ax1.set_aspect('equal')
ax1.set(xlim=(-1, 16), ylim=(-1, 16))
line, = ax1.plot([],[])

def animate(i):

    """perform animation step"""

    global x_goal, y_goal, theta_goal, x_traj, y_traj, theta_hist, dt

    x = x_traj[-1]
    y = y_traj[-1]
    theta = theta_hist[-1]
    plot_vehicle(x, y, theta, x_traj, y_traj)

    x_diff = x_goal - x
    y_diff = y_goal - y
    
    gamma = np.arctan2(y_diff, x_diff)
    d = np.sqrt(x_diff**2 + y_diff**2) # rho, np.linalg.norm(d_map_frame[:2])
    
    """
    Restrict alpha and beta (angle differences) to the range
    [-pi, pi] to prevent unstable behavior e.g. difference going
    from 0 rad to 2*pi rad with slight turn
    """
    alpha = wrap_angle(theta_goal - theta) # theta_diff

    v = Kp_rho * d
    
    if alpha == 0:
        w = 0
    else:
        R = 0.5 * d / np.sin(alpha/2)
        w = v / R  # must be depended on v such way so path becomes an arc

    beta = wrap_angle(gamma - alpha/2)

    vx = v * np.cos(beta)
    vy = v * np.sin(beta)

#     print('x=', x)
#     print('y=', y)
#     print('vx=', vx)
#     print('vy=', vy)
#     print('vx * dt', vx * dt)
#     print('vy * dt', vy * dt)
    
#     print('__________________________')
    
    theta += w * dt
    x += vx * dt
    y += vy * dt

    x_traj.append(x)
    y_traj.append(y)
    theta_hist.append(theta)

    line.set_data(x, y)

def transformation_matrix(x, y, theta):
    return np.array([[np.cos(theta), -np.sin(theta), x],
                     [np.sin(theta), np.cos(theta), y],
                     [0, 0, 1]])

def wrap_angle(angle):
    """
    Wraps the given angle to the range [-pi, +pi].

    :param angle: The angle (in rad) to wrap (can be unbounded).
    :return: The wrapped angle (guaranteed to in [-pi, +pi]).
    """

    pi2 = 2 * np.pi

    while angle < -np.pi:
        angle += pi2

    while angle >= np.pi:
        angle -= pi2
    
    return angle

def plot_vehicle(x, y, theta, x_traj, y_traj):
    # Corners of triangular vehicle when pointing to the right (0 radians)

    ax1.cla()

    p1_i = np.array([0.5, 0, 1]).T
    p2_i = np.array([-0.5, 0.25, 1]).T
    p3_i = np.array([-0.5, -0.25, 1]).T

    T = transformation_matrix(x, y, theta)
    p1 = np.matmul(T, p1_i)
    p2 = np.matmul(T, p2_i)
    p3 = np.matmul(T, p3_i)

    ax1.plot([p1[0], p2[0]], [p1[1], p2[1]], 'k-')
    ax1.plot([p2[0], p3[0]], [p2[1], p3[1]], 'k-')
    ax1.plot([p3[0], p1[0]], [p3[1], p1[1]], 'k-')

    ax1.plot(x_traj, y_traj, 'b--')

    ax1.arrow(x_start, y_start, np.cos(theta_start),
          np.sin(theta_start), color='r', width=0.1)
    ax1.arrow(x_goal, y_goal, np.cos(theta_goal),
          np.sin(theta_goal), color='g', width=0.1)

    ax1.set_aspect('equal')
    ax1.set(xlim=(-1, 16), ylim=(-1, 16))


ani_arc = FuncAnimation(fig, animate, interval = 10, frames=100)

In [3]:
ani_arc